In [1]:
import numpy as np
import pygame as pg
import time

pygame 2.6.1 (SDL 2.28.4, Python 3.12.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
def run(screen):
    running = True
    clock = pg.time.Clock()
    pg.display.flip()
    screen.fill('black')
    for event in pg.event.get():
        if event.type == pg.quit:
            running = False
        
    keys = pg.key.get_pressed()
    if keys[pg.K_ESCAPE] or keys[pg.K_q]:
        running = False
        
    dt = clock.tick(60)

    return running

def window_init(size=(1280,720)):
    pg.init()
    screen = pg.display.set_mode(size)
    font = pg.font.SysFont('Arial', 24)
    return screen, font

def handle_input():
    for event in pg.event.get():
        if event.type == pg.KEYDOWN:
            keys = pg.key.get_pressed()
            if keys[pg.K_d]:
                return 1
            elif keys[pg.K_a]:
                return -1
    return 0



In [130]:
# point movement functions

def dist(p1, p2):
    return float(np.linalg.norm(np.subtract(p1, p2)))

def get_angle(v1, v2):
    # convert to 3d vector, 2d only is depricated in numpy
    v1 = (v1[0], v1[1], 0)
    v2 = (v2[0], v2[1], 0)
    angle = np.arctan2(np.cross(v1, v2),np.dot(v1, v2))
    return float(angle[2]*180/np.pi)


def scale_points(points, x_scale, y_scale):
    new_points = []
    for point in points:
        new_points.append((point[0]*x_scale, point[1]*y_scale))
    return tuple(new_points)

def translate_points(points, offset):
    new_points = []
    for point in points:
        new_points.append(np.add(point, offset))
    return tuple(new_points)

def rotate_points(points, angle):
    angle = -angle*np.pi/180
    r_matrix = ((np.cos(angle), -np.sin(angle)),
                (np.sin(angle), np.cos(angle)))
    new_points = np.matmul(points, r_matrix)
    return tuple(new_points)

def squish_points(points, start, end):

    angle = get_angle(np.subtract(points[-1], points[0]), np.subtract(end, start))
    rotated_points = rotate_points(points, angle)

    points_start = rotated_points[0]
    points_end = rotated_points[-1]

    scale = dist(start, end)/dist(points_start, points_end)
    new_points = np.multiply(rotated_points, scale)

    new_points = np.add(new_points, np.subtract(start, new_points[0]))

    return tuple(new_points)

def get_segments(points):
    segments = []
    for i in range(0, len(points)-1):
        segments.append((points[i], points[i+1]))
    return tuple(segments)

In [4]:
# Drawing functions

def draw_points(points, surface=pg.display.get_surface(),color='white',width=2):
    pg.draw.lines(surface=surface,color=color,closed=False,points=points,width=width)

def prepare_points(points, surface=pg.display.get_surface()):
    x_width = surface.get_size()[0]
    y_width = surface.get_size()[1]

    x_points = []
    y_points = []
    for point in points:
        x_points.append(point[0])
        y_points.append(point[1])

    # translate points to make center of points the origin
    offset = (-(max(x_points)+min(x_points))/2, -(max(y_points)+min(y_points))/2)
    points = translate_points(points, offset)

    # flip y value of points, and scale points

    scale = min(x_width/(max(x_points)-min(x_points)),y_width/(max(y_points)-min(y_points)))*0.9
    points = scale_points(points=points, x_scale=scale, y_scale=(scale*-1))

    # translate points to origin of surface
    
    offset = (x_width/2, y_width/2)
    points = translate_points(points, offset)

    return points

In [5]:
def generate_fractal_points(points, iterations, timeout=5):
    fractal_points = [points]
    start_time = time.time()
    for i in range(0, iterations):
        new_points = []
        for s, segment in enumerate(get_segments(fractal_points[i])):
            if time.time()-start_time > timeout:
                raise(Exception('Fractal Generator Timed Out'))
            for p, point in enumerate(squish_points(points, segment[0], segment[1])):
                if s != 0 and p == 0:
                    # skip adding a point if its a duplicate
                    pass
                else:
                    new_points.append(point)
        fractal_points.append(np.array(new_points))
    return fractal_points

In [ ]:
'''
Old method, new one is waaaay faster

def generate_fractal_points(points, iterations, timeout=5):
    fractal_points = [points]
    start_time = time.time()
    for i in range(0, iterations):
        new_points = []
        for segment in get_segments(fractal_points[i]):
            if time.time()-start_time > timeout:
                raise(Exception('Fractal Generator Timed Out'))
            for point in squish_points(points, segment[0], segment[1]):
                new_points.append(point)
        fractal_points.append(np.array(new_points))
    return fractal_points
'''

In [6]:
def generate_fractal_points(points, iterations, timeout=5):
    fractal_points = [points]
    start_time = time.time()
    for i in range(0, iterations):
        new_points = []
        for segment in get_segments(points):
            if time.time()-start_time > timeout:
                raise(Exception('Fractal Generator Timed Out'))
            for point in squish_points(fractal_points[i], segment[0], segment[1]):
                new_points.append(point)
        fractal_points.append(np.array(new_points))
    return fractal_points

In [106]:
# dragon something

points = np.array(
    ((-1, 0),
    (0, 0),
    (0, 1))
)

n = 15

In [115]:
# koch snowflake

points = np.array(
    ((-1.5, 0),
    (-0.5, 0),
    (0, np.sqrt(3)/2),
    (0.5, 0),
    (1.5, 0))
)

n = 5

In [125]:
# chain thing

points = np.array(
    ((-0.5, 1),
    (0.5, 1),
    (0.5, 0),
    (-0.5, 0),
    (-0.5, -1),
    (0.5, -1))
)

n = 6

In [126]:
# chain thing 2

points = np.array(
    ((-2.5, 0),
    (-1.5, 0),
    (-1.5, 1),
    (-0.5, 1),
    (0.5, 1),
    (0.5, 0),
    (-0.5, 0),
    (-0.5, -1),
    (0.5, -1),
    (1.5, -1),
    (1.5, 0),
    (2.5, 0)
    )
)

n = 4

In [511]:
# dragon pattern 2 i guess

points = (
    (-0.5, 1),
    (-0.5, 0),
    (0.5, 0),
    (0.5, -1)
)

n = 6

In [132]:
# off koch snowflake or something

points = (
    (-1.5, 0),
    (-0.5, 0),
    (0, 1.3),
    (0.5, 0),
    (1.5, 0)
)

n = 7

In [421]:
# pyramid

points = (
    (-1.5, 0),
    (-0.5, 0),
    (-0.5, 1),
    (0.5, 1),
    (0.5, 0),
    (1.5, 0)
)

n = 5

In [481]:
# thing

points = (
    (-0.5, -1),
    (0.5, -1),
    (0.5, 0),
    (-0.5, 0),
    (-0.5, 1)
)

points = rotate_points(points, -np.arctan(0.5)*180/np.pi)

n = 9

In [508]:
# thing evened out

points = (
    (-0.5, -1),
    (0.5, -1),
    (0.5, 0),
    (-0.5, 0),
    (-0.5, 0.75)
)

points = rotate_points(points, -np.arctan(0.5*2/1.75)*180/np.pi)

n = 9

In [410]:
# w FOR WUMBO

points = (
    (-1.5, 0),
    (-1, 0.5),
    (-0.5, 0),
    (0, -0.5),
    (0.5, 0),
    (1, 0.5),
    (1.5, 0)
)

n = 5

In [478]:
# hex chain thing

points = (
    (-1.5, np.sqrt(3)/2),
    (-1, np.sqrt(3)),
    (0, np.sqrt(3)),
    (0.5, np.sqrt(3)/2),
    (0, 0),
    (-0.5, -np.sqrt(3)/2),
    (0, -np.sqrt(3)),
    (1, -np.sqrt(3)),
    (1.5, -np.sqrt(3)/2)
)

n = 5

In [472]:
# the claw

points = (
    (0, 1),
    (-1, 1),
    (-1, 0),
    (-1, -1),
    (0, -1),
    (1, -1)
)

n = 7

In [469]:
# the claw 2

points = (
    (0, 1),
    (-1, 1),
    (-1, 0),
    (-1, -1),
    (0, -1),
    (0, 0),
    (1, 0),
    (1, -1)
)

n = 6

In [466]:
# the claw 2 i guess

points = (
    (-1.25, -np.sqrt(3)/4),
    (-0.75, np.sqrt(3)/4),
    (0.25, np.sqrt(3)/4),
    (-0.25, -np.sqrt(3)/4),
    (0.75, -np.sqrt(3)/4),
    (1.25, np.sqrt(3)/4),
)

n = 6

In [463]:
# hex scary 1

points = (
    (-1, 0),
    (-0.5, np.sqrt(3)/2),
    (0.5, np.sqrt(3)/2), 
    (1, 0)
)

n = 11

In [460]:
# hex scary 2

points = (
    (-0.5, -np.sqrt(3)/2),
    (-1, 0),
    (-0.5, np.sqrt(3)/2),
    (0.5, np.sqrt(3)/2), 
    (1, 0)
)

n = 8

In [442]:
# kramers shard 1

points = (
    (-1, 0),
    (-1, 1),
    (0, 0),
    (1, 0)
)

n = 11

In [142]:
# kramers shard 2

points = (
    (-1, 0),
    (-1, 1),
    (0, 0),
    (1, 0)
)

n = 11

In [139]:
# kramer wave 1

points = (
    (-1, 0.5),
    (-1, 1),
    (0, 0),
    (1, 0)
)

n = 12

In [554]:
# the crevasse

points = (
    (-1, 1),
    (-1, 0),
    (1, 0),
    (1, 1)
)

n = 11

In [433]:
# doom helmet 1

points = (
    (-1, 0),
    (-1, 1),
    (0, 1.6),
    (1, 1),
    (1, 0)
)

n = 9

In [557]:
# tree 1

points = (
    (-1.5, 0),
    (-1, np.sqrt(3)/2),
    (1, np.sqrt(3)/2),
    (1.5, 0)
)

n = 9

In [560]:
# razor 1

points = (
    (-1, 0),
    (-1, 1),
    (0, 0.5),
    (1, 0)
)

n = 10

In [563]:
# rock golem 1

points = (
    (-2, 0),
    (-1, 2),
    (0, 4),
    (1, 2),
    (2, 0)
)

n = 8

In [136]:
# hex thing A

points = (
    (-2, 0),
    (-1, np.sqrt(3)),
    (0, np.sqrt(3)*2),
    (1, np.sqrt(3)),
    (2, 0)
)

n = 10

In [94]:
# bird 1

points = (
    (-2, 2),
    (-1, 1.5),
    (0, 0),
    (1, 1.5),
    (2, 2)
)

n = 8

In [97]:
# necklace 1

points = (
    (-2, 1),
    (-1, 1.5),
    (0, 0),
    (1, 1.5),
    (2, 1)
)

n = 8

In [517]:
# the u

points = (
    (-1, 1),
    (-1, 0),
    (0, 0),
    (1, 0),
    (1, 1)
)

n = 9

In [569]:
# the flat hat

points = (
    (-1.5, 0),
    (-0.5, 0),
    (-0.5, 1),
    (0.5, 1),
    (0.5, 0),
    (1.5, 0)
)

n = 7

In [91]:
# the yea bro 

points = (
    (-1, 0),
    (-1, -1),
    (0, -1),
    (0, 0),
    (0.75, 0)
)

n = 9

In [143]:
fractal_points = generate_fractal_points(points, n, 10)

In [144]:
screen, font = window_init((1600,1000))
new_fractal_points = []
for points in fractal_points:
    new_fractal_points.append(np.array(prepare_points(points, screen)))

number = 0

while run(screen):
    draw_points(surface=screen, points=new_fractal_points[number], width=1)
    number = number + handle_input()
    if number > len(new_fractal_points)-1:
        number = len(new_fractal_points) - 1
    if number < 0:
        number = 0
    text = font.render(str(number), True, 'white')
    screen.blit(text, (0,0))
pg.quit()

In [245]:
pg.quit()